## Importation des données et création des sous-corpus pour l'analyse

In [23]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from collections import defaultdict
import tensorflow as tf
from transformers import pipeline, TFDistilBertForSequenceClassification, DistilBertTokenizer



In [24]:
#Récupération des données annotées
tabA=pd.read_csv("edos_labelled_aggregated.csv",sep=",")

# Création d'un fichier avec les entrées non annotés
with open("edos_non_labelled_aggregated.csv", "w", encoding="utf-8") as f_out_total:
    f_out_total.write("rewire_id,text\n")
    for i in range(len(tabA)):
        id=tabA["rewire_id"].iloc[i]
        text = tabA["text"].iloc[i]
        text_escaped = text.replace('"', '""')
        f_out_total.write(id+",\""+text_escaped+"\"\n")

# Création d'un fichier avec le texte des entrées sexistes et un fichier avec le texte des entrées non sexistes
with open("non_labelled_sexist.csv", "w", encoding="utf-8") as f_out_sexist, open("non_labelled_nonsexist.csv", "w", encoding="utf-8") as f_out_nonsexist:
    f_out_sexist.write("rewire_id,text\n")
    f_out_nonsexist.write("rewire_id,text\n")
    for j in range(len(tabA)):
        id=tabA["rewire_id"].iloc[j]
        label = tabA["label_sexist"].iloc[j]
        text = tabA["text"].iloc[j]
        text_escaped = text.replace('"', '""')
        if label == "not sexist":
            f_out_nonsexist.write(id+",\""+text_escaped + "\"\n")
        else:
            f_out_sexist.write(id+",\""+text_escaped + "\"\n")


## Première étape : Création d'une regex efficace pour repérer les termes sexistes

### Vérification de la présence d'une ou plusieurs références (sexistes et/ou descriptives) aux femmes dans toutes les entrées du sous-corpus sexiste

On cherche d'abord à ajouter le plus de mots à notre regex pour englober le plus de possibilité (et à récupérer les id des entrées qui ne matchent pas avec cette dernière pour pouvoir éventuellement les analyser par la suite)

In [30]:
liste_unmatched_Csexist=[]

# Motif regex le plus large (insultes, termes péjoratifs et termes génériques)
patternGeneral = r'\b(?!(?:man|men)\b)(she|her|\w*wh?[oayi]mm?[eayi]n?[sz]?|wi[vf]es?|girls?|lad[yi]e?s?|daughters?|sisters?|girlfriends?|aunti?e?s?|grandmothers?|mothers?|m[ou]mm?[iy]?e?s?|females?|feminine|puss[yi]?e?s?|\wiy?a?tche?s?y?(ing)?(tard)?|cunts?|whores?(dom)?(llywood)?|sluts?t?y?(iness)?|hoe?s?|slags?|metoo|femininity|feminis[mt]s?|(trad)?e?thots?(life)?(dom)?(ism)?|femoids?|harp[yi]e?s?|twats?|spinsters?|tranny|chicks?|roa?sties?|stac[y|i]e?s?|[v|h]ags?|vaginas?|(noodle)?foids?|feminazis?|d[yi]kes?|gals?|skanks?|gold digg(er)?(ing)?s?|bi--th)\b'

#Regex pour conserver la reférence de l'entrée
patternID = r'\b(sexism2022_english-\d+)\b'

# Compiler les regex (avec l'option d'insensibilité à la casse pour le motif général)
regexGeneral = re.compile(patternGeneral, re.IGNORECASE)
regexID = re.compile(patternID)

# Vérification de la présence d'au moins un terme de la regex générale dans le corpus sexiste
nbr_entrees_sexist=0
nbr_entrees_matchs=0
with open("non_labelled_sexist.csv", "r", encoding="utf-8") as f_in_sexist:
    for index,line in enumerate(f_in_sexist, start=1):
        if index>=2:
            nbr_entrees_sexist+=1
            # Trouver toutes les correspondances et les compter
            matches = regexGeneral.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1
            # On récupére les id des entrées non matchées
            else:
                unmatches = regexID.findall(line)
                # On peut afficher les entrées ne contenant pas de correspondances avec la regex actuelle afin de rajouter des termes sexistes repérables
                #print(line)
                liste_unmatched_Csexist.append(unmatches[0])

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur ", nbr_entrees_sexist,". On a donc réussi à matcher", nbr_entrees_matchs/nbr_entrees_sexist*100, "% des entrées sexistes avec cette regex.")

# Nous conservons les id des entrées non matchées pour pouvoir les étudier par la suite
print("Les id des entrées non matchées sont :", liste_unmatched_Csexist)
print("Nombre d'entrées non matchées :", len(liste_unmatched_Csexist))


Nombre d'entrées contenant au moins un terme de la regex : 4794 sur  4854 . On a donc réussi à matcher 98.76390605686032 % des entrées sexistes avec cette regex.
Les id des entrées non matchées sont : ['sexism2022_english-4332', 'sexism2022_english-16502', 'sexism2022_english-12602', 'sexism2022_english-2378', 'sexism2022_english-12024', 'sexism2022_english-4392', 'sexism2022_english-2930', 'sexism2022_english-9285', 'sexism2022_english-8155', 'sexism2022_english-3880', 'sexism2022_english-7046', 'sexism2022_english-3886', 'sexism2022_english-1069', 'sexism2022_english-18782', 'sexism2022_english-2013', 'sexism2022_english-6417', 'sexism2022_english-4167', 'sexism2022_english-19842', 'sexism2022_english-6159', 'sexism2022_english-9733', 'sexism2022_english-12463', 'sexism2022_english-3906', 'sexism2022_english-5216', 'sexism2022_english-12613', 'sexism2022_english-17748', 'sexism2022_english-2961', 'sexism2022_english-1028', 'sexism2022_english-2285', 'sexism2022_english-8980', 'sexism

On a donc une très bonne converture. Cependant, cette regex prend en compte n'importe quelle mention d'une personne féminine, peu importe la polarité de l'entrée.
Pour vérifier si elle peut tout de même être pertinente, nous allons la tester sur le corpus non sexiste.

In [28]:
nbr_entrees_nonsexist=0
nbr_entrees_matchs=0
with open("non_labelled_nonsexist.csv", "r", encoding="utf-8") as f_in_nonsexist:
    for index,line in enumerate(f_in_nonsexist, start=1):
        if index>=2:
            nbr_entrees_nonsexist+=1
            matches = regexGeneral.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur", nbr_entrees_nonsexist,". Cette regex matche donc également", nbr_entrees_matchs/nbr_entrees_nonsexist*100, "% des entrées non sexistes.")

Nombre d'entrées contenant au moins un terme de la regex : 13057 sur 15146 . Cette regex matche donc également 86.20757955895945 % des entrées non sexistes.


Même si l'on remarque une différence d'environ 13 points entre les 2 corpus, on voit que notre regex est bien trop large puisqu'elle matche une très grande majorité du corpus non sexiste, à cause notamment de la présence de termes génériques comme "she" et "her" par exemple. Nous allons donc tenter d'affiner celle-ci en ne retenant que les termes sexistes ou faisant référence au féminisme.

Cette regex "Insults" a été définie en retenant les insultes à proprement parler, les mentions de feminisme, féminité ou des féministes, ainsi que les variations orthographiques du terme "woman" ou "women" (mais sans leur version correctement orthographiées) puisque c'est une technique régulièrement utilisée sur les réseaux sociaux pour parler d'une population de manière discriminante tout en étant plus difficilement repérable par les modérateur.ice.s des sites.

In [31]:
liste_unmatched_insults_Csexist=[]

# Motif regex avec les insultes et mention du féminisme
patternInsults= r'\b(?!(?:wo)?man\b|men\b|women\b)(wh?[oai]mm?[eayi]*n?[sz]?|females?|puss[yi]?e?s?|cunts?|\wiy?a?tche?y?s?(ing)?(tard)?|whores?(dom)?(llywood)?|sluts?t?y?(iness)?|hoe?s?|slags?|metoo|femininity|feminis[mt]s?|feminine|e?(trad)?thots?(life)?(ism)?|femoids?|harp[yi]e?s?|twats?|spinsters?|tranny|chicks?|roa?sties?|stac[yi]e?s?|[vh]ags?|vaginas?|(noodle)?foids?|feminazis?|d[yi]kes?|gals?|skanks?|gold digg(er)?(ing)?s?|bi--th|slutty)\b'
regexInsults = re.compile(patternInsults, re.IGNORECASE)

# Vérification de la présence d'au moins un terme insultant dans les entrées sexistes
nbr_entrees_matchs=0
with open("non_labelled_sexist.csv", "r", encoding="utf-8") as f_in_sexist:
    for index,line in enumerate(f_in_sexist, start=1):
        if index>=2:
            matches = regexInsults.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur", nbr_entrees_sexist,". On a donc réussi à matcher", nbr_entrees_matchs/nbr_entrees_sexist*100, "% des entrées sexistes avec cette regex plus précise.")


Nombre d'entrées contenant au moins un terme de la regex : 2444 sur 4854 . On a donc réussi à matcher 50.3502266172229 % des entrées sexistes avec cette regex plus précise.


On perd près de 50 points par rapport à la regex générale. Comparons cependant cette nouvelle regex avec le corpus non sexiste.

In [32]:
# Vérification de la présence d'au moins un terme insultant dans les entrées non sexistes
nbr_entrees_matchs=0
with open("non_labelled_nonsexist.csv", "r", encoding="utf-8") as f_in_nonsexist:
    for index,line in enumerate(f_in_nonsexist, start=1):
        if index>=2:
            matches = regexInsults.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur", nbr_entrees_nonsexist,". Cette regex matche donc également", nbr_entrees_matchs/nbr_entrees_nonsexist*100, "% des entrées non sexistes.")

Nombre d'entrées contenant au moins un terme de la regex : 1242 sur 15146 . Cette regex matche donc également 8.200184867291695 % des entrées non sexistes.


Si l'on retrouve tout de même dans 8% des entrées non sexistes des termes de notre regexInsults, on a réduit de 78 points notre résultat par rapport à la regexGeneral. On peut essayer de diminuer encore ce nombre en enlevant les termes "feminist(s)", "feminism", "metoo", "feminine", "female(s)", "gal(s)", "chick(s)", "vagina(s)" et "femininity" qui ne sont pas des termes insultants en soi, ainsi que l'expression "witch hunt", afin d'obtenir une regex très spécifique pour laquelle on serait presque certains que la présence d'un des termes de celle-ci dans une entrée la classe automatiquement comme une entrée sexiste.

In [33]:
# Création de notre regex la plus spécifique
patternInsultsSpe= r'\b(?!(?:wo)?man\b|men\b|women\b|witch[-\s]hunts?\b)(wh?[oaiy]mm?[eayi]*n?[sz]?|puss[yi]?e?s?|cunts?|\wiy?a?tche?y?s?(ing)?(tard)?|whores?(dom)?(llywood)?|slut(tiness)?s?(ting)?|hoe?s?|slags?|(trad)?e?thots?(ism)?(dom)?(life)?|femoids?|harp[yi]e?s?|twats?|spinsters?|tranny|roa?sties?|stac[yi]e?s?|[vh]ags?|(noodle)?foids?|feminazis?|d[yi]kes?|skanks?|gold digg(er)?s?|bi--th|slutty)\b'
regexInsultsSpe = re.compile(patternInsultsSpe, re.IGNORECASE)

nbr_entrees_matchs=0
with open("non_labelled_nonsexist.csv", "r", encoding="utf-8") as f_in_nonsexist:
    for index,line in enumerate(f_in_nonsexist, start=1):
        if index>=2:
            matches = regexInsultsSpe.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1


print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur", nbr_entrees_nonsexist,". Cette regex matche également", nbr_entrees_matchs/nbr_entrees_nonsexist*100, "% des entrées non sexistes.")

Nombre d'entrées contenant au moins un terme de la regex : 257 sur 15146 . Cette regex matche également 1.6968176416215501 % des entrées non sexistes.


Notre regexInsultSpe semble donner assez peu de faux positif. Vérifions cependant la quantité d'entrées sexistes contenant au moins un mot de notre regexInsultsSpe afin d'observer si on obtiendrait beaucoup de faux négatifs avec seulement cette méthode.



In [34]:
liste_sexist_unmatchedRegexSpe = []
# Vérification de la présence d'au moins un terme insultant spécifique dans les entrées sexistes
nbr_entrees_matchs=0
with open("non_labelled_sexist.csv", "r", encoding="utf-8") as f_in_sexist:
    for index,line in enumerate(f_in_sexist, start=1):
        if index>=2:
            matches = regexInsultsSpe.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1
            # Afficher les entrées ne contenant pas de correspondances afin de rajouter des termes à la regex et récupérer les id des entrées non matchées
            else:
                unmatches = regexID.findall(line)
                liste_sexist_unmatchedRegexSpe.append(unmatches[0])

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur", nbr_entrees_sexist,". On a donc réussi à matcher", nbr_entrees_matchs/nbr_entrees_sexist*100, "% des entrées sexistes avec notre regex la plus précise.")
print("Les id des entrées non matchées avec notre regex la plus spécifique sont :", liste_sexist_unmatchedRegexSpe)
print("Nombre d'entrées non matchées avec notre regex la plus précise :", len(liste_sexist_unmatchedRegexSpe))

Nombre d'entrées contenant au moins un terme de la regex : 1933 sur 4854 . On a donc réussi à matcher 39.82282653481665 % des entrées sexistes avec notre regex la plus précise.
Les id des entrées non matchées avec notre regex la plus spécifique sont : ['sexism2022_english-7228', 'sexism2022_english-3553', 'sexism2022_english-14992', 'sexism2022_english-7260', 'sexism2022_english-6629', 'sexism2022_english-7022', 'sexism2022_english-10268', 'sexism2022_english-12319', 'sexism2022_english-5581', 'sexism2022_english-2771', 'sexism2022_english-19849', 'sexism2022_english-15676', 'sexism2022_english-9087', 'sexism2022_english-15184', 'sexism2022_english-18377', 'sexism2022_english-13992', 'sexism2022_english-14297', 'sexism2022_english-704', 'sexism2022_english-11647', 'sexism2022_english-8721', 'sexism2022_english-11296', 'sexism2022_english-15860', 'sexism2022_english-8476', 'sexism2022_english-3673', 'sexism2022_english-13224', 'sexism2022_english-12662', 'sexism2022_english-9301', 'sexi

On peut ainsi voir que, même si l'on ne dépasse pas le 40% du corpus sexiste, la présence d'au moins un terme de notre regexInsultSpe est relativement efficace comme premier filtre. La recherche de match avec notre regex la plus spécifique sera donc notre première étape de catégorisation.

# Deuxième étape : "Nettoyage" du corpus et regex descriptive

Pour ne pas repasser sur les entrées sexistes déjà "reconnues" comme telles, nous créons un sous corpus avec uniquement les entrées sexistes qui passent le premier filtre.

In [35]:
tabS=pd.read_csv("non_labelled_sexist.csv",sep=",")

# Création d'un fichier avec les entrées sexistes qui ne matchent pas notre regex la plus spécifique
with open("non_labelled_sexist_unmatchedRegexSpe.csv", "w", encoding="utf-8") as f_out_unmatched:
    f_out_unmatched.write("rewire_id,text\n")
    for i in range(len(tabS)):
        id=tabS["rewire_id"].iloc[i]
        text = tabS["text"].iloc[i]
        for j in liste_sexist_unmatchedRegexSpe:
            if id == j:
                text_escaped = text.replace('"', '""')
                f_out_unmatched.write(id+",\""+text_escaped+"\"\n")

tabUmRS=pd.read_csv("non_labelled_sexist_unmatchedRegexSpe.csv",sep=",")
print("Nous avons ainsi bien recupéré les",len(tabUmRS),"entrées sexistes qui ne matchent pas notre regex la plus spécifique.")

Nous avons ainsi bien recupéré les 2921 entrées sexistes qui ne matchent pas notre regex la plus spécifique.


Puisque nous savons que nous ne trouverons pas d'insultes sexistes déjà repertorié dans ce sous corpus mais que des termes plus génériques liés aux femmes sont présents (puisque notre première regex contenait ces derniers et qu'elle couvrait plus de 99% du corpus sexiste), nous recherchons ces termes dans notre corpus. Nous réintégrons également les termes comme "vagina(s)", "chick(s) ou "gal(s)".

In [36]:
liste_unmatchedRegexDsc=[]
# Motif regex avec les termes génériques
patternDsc = r'\b(?!(?:man|men)\b)(she|her|\w*wom[ae]n|wi[v|f]es?|girls?|lad[y|i]e?s?|daughters?|sisters?|girlfriends?|aunti?e?s?|grandmothers?|mothers?|m[o|u]mm?[i|y]?e?s?|females?|feminine|metoo|femininity|feminis[m|t]s?|gals?|chicks?|vaginas?)\b'

# Compiler les regex (avec l'option d'insensibilité à la casse pour le motif descriptif)
regexDsc = re.compile(patternDsc, re.IGNORECASE)

# Vérification de la présence d'au moins un terme de la regex générale dans le corpus sexiste
nbr_entrees=0
nbr_entrees_matchs=0
with open("non_labelled_sexist_unmatchedRegexSpe.csv", "r", encoding="utf-8") as f_in_sexistNonInsults:
    for index,line in enumerate(f_in_sexistNonInsults, start=1):
        if index>=2:
            nbr_entrees+=1
            # Trouver toutes les correspondances
            matches = regexDsc.findall(line)
            if len(matches) > 0:
                nbr_entrees_matchs+=1
            # On récupére les id des entrées non matchées
            else:
                unmatches = regexID.findall(line)
                liste_unmatchedRegexDsc.append(unmatches[0])

print("Nombre d'entrées contenant au moins un terme de la regex :", nbr_entrees_matchs, "sur ", nbr_entrees,". On a donc réussi à matcher", nbr_entrees_matchs/nbr_entrees*100, "% des entrées sexistes avec cette regex.")

# Nous conservons les id des entrées non matchées pour pouvoir les étudier par la suite
print("Les id des entrées non matchées sont :", liste_unmatchedRegexDsc)
print("Leur nombre est de", len(liste_unmatchedRegexDsc))

Nombre d'entrées contenant au moins un terme de la regex : 2862 sur  2921 . On a donc réussi à matcher 97.98014378637453 % des entrées sexistes avec cette regex.
Les id des entrées non matchées sont : ['sexism2022_english-4332', 'sexism2022_english-16502', 'sexism2022_english-12602', 'sexism2022_english-2378', 'sexism2022_english-12024', 'sexism2022_english-4392', 'sexism2022_english-2930', 'sexism2022_english-9285', 'sexism2022_english-8155', 'sexism2022_english-3880', 'sexism2022_english-7046', 'sexism2022_english-3886', 'sexism2022_english-1069', 'sexism2022_english-18782', 'sexism2022_english-2013', 'sexism2022_english-6417', 'sexism2022_english-4167', 'sexism2022_english-19842', 'sexism2022_english-6159', 'sexism2022_english-9733', 'sexism2022_english-12463', 'sexism2022_english-3906', 'sexism2022_english-5216', 'sexism2022_english-12613', 'sexism2022_english-17748', 'sexism2022_english-2961', 'sexism2022_english-1028', 'sexism2022_english-2285', 'sexism2022_english-8980', 'sexism

# Troisième étape : Evaluation de la polarité et classification sexiste/non sexiste grâce à des modèles Transformers

Nous allons maintenant utiliser un modèle Transformers afin d'étudier les entrées contenant au moins un terme faisant référence à une ou des femmes mais ne contenant pas les termes insultants que nous avions mis dans notre regexInsultsSpe. Pour ce faire, nous avons choisi de classer toutes nos entrées selon leur polarité.
Ce premier classement nous permettra de voir la quantité de données perdues avec cette méthode.
Si cela est efficace et ne nous fait pas perdre trop de faux négatif, nous procèderons à la même analyse sur le corpus non sexiste.


In [30]:
compteur_pola_pos=0
compteur_pola_neg=0
compteur_pola_neutre=0

# Charger le tokenizer et le modèle
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)

with open("non_labelled_sexist_unmatchedRegexSpe.csv", "r", encoding="utf-8") as f_in_sexistNonInsults, open("non_labelled_sexist_unmatchedRegexSpe_neg.csv", "w", encoding="utf-8") as f_out_sexist_neg, open("non_labelled_sexist_unmatchedRegexSpe_pos.csv", "w", encoding="utf-8") as f_out_sexist_pos, open("non_labelled_sexist_unmatchedRegexSpe_neutre.csv", "w", encoding="utf-8") as f_out_sexist_neutre:
    f_out_sexist_neg.write("rewire_id,text\n")
    f_out_sexist_pos.write("rewire_id,text\n")
    f_out_sexist_neutre.write("rewire_id,text\n")
    for index,line in enumerate(f_in_sexistNonInsults, start=1):
        if index>=2:
            matches = regexDsc.findall(line)
            if len(matches) > 0:
                inputs = tokenizer(line, return_tensors="tf", padding=True, truncation=True)
                # Effectuer des prédictions
                outputs = model(inputs.data)
                # Obtenir les scores et les étiquettes prédites
                predictions = tf.nn.softmax(outputs.logits, axis=-1)
                predicted_labels = tf.argmax(predictions, axis=1)
                # Définir le mapping des labels
                label_map = {0: 'positive', 1: 'neutral', 2: 'negative'}
                # Filtrer et afficher les résultats
                for text, label in zip(line, predicted_labels):
                    label_str = label_map[label.numpy()]
                    if label_str in ['positive']:
                        f_out_sexist_pos.write(line)
                        compteur_pola_pos+=1
                    elif label_str in ['negative']:
                        f_out_sexist_neg.write(line)
                        compteur_pola_neg+=1
                    elif label_str in ['neutral']:
                        f_out_sexist_neutre.write(line)
                        compteur_pola_neutre+=1

print("Le nombre d'entrées sexistes avec un polarité positive est de", compteur_pola_pos, "soit", compteur_pola_pos/len(tabUmRS)*100, "%.")
print("Le nombre d'entrées sexistes avec une polarité négative est de", compteur_pola_neg, "soit", compteur_pola_neg/len(tabUmRS)*100, "%.")
print("Le nombre d'entrées sexistes avec une polarité neutre est de", compteur_pola_neutre, "soit", compteur_pola_neutre/len(tabUmRS)*100, "%.")

                

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Le nombre d'entrées sexistes positives est de 376 soit 12.859097127222983 %.
Le nombre d'entrées sexistes négatives est de 2479 soit 84.78112175102599 %.
Le nombre d'entrées sexistes neutres est de 27 soit 0.9233926128590971 %.


Au vu du temps d'exécution assez important (près de 15 minutes pour un corpus de moins de 5000 entrées), nous avons fait le choix de ne pas poursuivre dans cette voie puisque notre corpus non sexiste comprend plus de 15000 entrées. Nous avons tout de même indiquer en NB 2 et dans le dossier ce que nous aurions aimé mettre en place si notre système le permettait.

Si cette méthode semble relativement efficace puisque nous perdons assez peu d'entrées (environ 13% de polarité positive, soit des faux négatif), non avons conscience que ne pas pouvoir vérifier son efficacité sur le corpus non sexiste est un biais important pour nos résultats.

# NB : Pistes de réflexion et/ou d'améliorations

## NB1 : Catégories mal/non reconnues grâce à l'analyse lexicale proposée

On revient rapidement ici sur notre liste d'entrées sexistes pour laquelle notre regexGeneral n'était pas efficace et nous allons chercher à savoir si elles correspondent à un ou des vecteurs particuliers de sexisme que l'on ne peut pas repérer avec une analyse lexicale grâce à un dictionnaire. (Nous avons choisi les vecteurs plutôt que les catégories pour être le plus précis possible.)

In [37]:
# Création d'un dictionnaire pour compter et classer les vector_category
vector_counts = defaultdict(int)

#On récupère les label_category des entrées non matchées et on les stocke dans un dictionnaire pour les compter

for i in range(len(tabA)):
    id=tabA["rewire_id"].iloc[i]
    label_vector = tabA["label_vector"].iloc[i]
    text = tabA["text"].iloc[i]
    for j in liste_unmatched_Csexist:
        if id == j:
            vector_counts[label_vector] += 1

# On trie les label_category par ordre décroissant
vector_counts = dict(sorted(vector_counts.items(), key=lambda item: item[1], reverse=True))

# On affiche la répartition des vector_category
for vector, count in vector_counts.items():
    percentage = (count / len(liste_unmatched_Csexist)) * 100
    print(f"{vector}: {count} ({percentage:.2f}%)")


2.1 descriptive attacks: 11 (18.33%)
3.2 immutable gender differences and gender stereotypes: 10 (16.67%)
4.2 supporting systemic discrimination against women as a group: 9 (15.00%)
2.3 dehumanising attacks & overt sexual objectification: 8 (13.33%)
3.1 casual use of gendered slurs, profanities, and insults: 7 (11.67%)
1.2 incitement and encouragement of harm: 5 (8.33%)
3.3 backhanded gendered compliments: 3 (5.00%)
2.2 aggressive and emotive attacks: 3 (5.00%)
4.1 supporting mistreatment of individual women: 2 (3.33%)
3.4 condescending explanations or unwelcome advice: 1 (1.67%)
1.1 threats of harm: 1 (1.67%)


Les deux catégories les plus représentées sont donc "3.2 immutable gender differences and gender stereotypes" et "2.1 descriptive attacks". Il serait intéressant d'analyser plus précisément ces entrées pour repérer leurs caractéristiques afin de faciliter leurs catégorisation.

## NB 2 Suite possible du projet

Si nous avions poussé plus loin le projet, nous aurions procédé comme suit :

- Nous aurions tout d'abord vérifié le taux de couverture sur le corpus non sexiste.
La première étape aurait été de filtrer les entrées avec notre regex descriptive pour vérifier la présence d'au moins un terme faisant référence aux femmes puis la répartition des polarités sur ces entrées.

- Si le résultat obtenu avait montré que la polarité était majoritairement neutre ou positive dans le corpus non sexiste, nous aurions considéré que l'association de notre regex et de l'analyse de la polarité était une méthode efficace.
Dans le cas contraire, nous aurions souhaité utiliser un modèle de classification en 2 classes : sexiste/non-sexiste comme le modèle 'NLP-LTU/distilbert-sexism-detector'qui obtient de bons résultats pour cette tâche.